In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import duckdb
import requests
import json
import os
from dotenv import load_dotenv
import logging
from src.imports import get_stops
from src.data_processing import extract_stop_info, normalize_stops

load_dotenv()

True

In [3]:
stop_names = [
    "Hradčanská", "Sparta", "Korunovační", "Letenské náměstí",
    "Kamenická", "Strossmayerovo náměstí", "Nábřeží Kapitána Jaroše",
    "Vltavská", "Výstaviště", "Veletržní palác", "Chotkovy sady",
]
base_url = 'https://api.golemio.cz/v2/gtfs/stops'

In [4]:
letna_stops = get_stops(base_url, os.getenv('GOLEMIO_TOKEN'), stop_names)

In [5]:
stops_data_df = pd.DataFrame(extract_stop_info(letna_stops))

In [40]:
df = normalize_stops(stops_data_df)
df.to_csv('letna_stops.csv')

In [7]:
import folium
from folium.plugins import MarkerCluster, HeatMap

In [8]:
m = folium.Map(location=[df['avg_latitude'].mean(), df['avg_longitude'].mean()], zoom_start=14, tiles="CartoDB Positron")

marker_cluster = MarkerCluster().add_to(m)

# Add each stop to the map with modern, tooltip-only markers
for _, stop in df.iterrows():
    folium.CircleMarker(
        location=[stop["avg_latitude"], stop["avg_longitude"]],
        radius=8,  # Size of the circle marker
        color="#FF5733",  
        fill=True,
        fill_color="#FF5733",  
        fill_opacity=0.7,  # Fill transparency
        tooltip=stop["stop_name"],  # Tooltip displayed on hover
        popup=(f"<b>Stop Name:</b> {stop['stop_name']}<br>"
               f"<b>Base Stop ID:</b> {stop['base_stop_id']}<br>"
               f"<b>All Stop IDs:</b> {', '.join(stop['all_stop_ids'])}")
    ).add_to(marker_cluster)

m

In [13]:
def setup_azure_duckdb(duckdb_conn: duckdb.DuckDBPyConnection) -> None:
    duckdb_conn.sql(f"ATTACH 'public-transport.db';")
    duckdb_conn.sql('INSTALL azure; LOAD azure;')
    duckdb_conn.sql(f'''
        CREATE SECRET azure_spn (
            TYPE AZURE,
            PROVIDER SERVICE_PRINCIPAL,
            TENANT_ID '{os.getenv("parquetAzureTenantID")}',
            CLIENT_ID '{os.getenv("parquetAzureAppID")}',
            CLIENT_SECRET '{os.getenv("parquetAzureClientSecret")}',
            ACCOUNT_NAME '{os.getenv("parquetStorageName")}');
            ''')
    logging.info("Azure and DuckDB setup complete")
    
def get_stop_times_from_azure(duckdb_conn, stop_ids) -> pd.DataFrame:
    stop_times_sql_string = f'''
    SELECT * 
        FROM 'azure://golem-data-lake-pid/vehiclepositions_stop_times_history/*/*/*/*.parquet'
        WHERE YEAR = 2024 AND gtfs_stop_id IN ({stop_ids})
    '''
    logging.info("Getting stop times from Azure")
    return duckdb_conn.sql(stop_times_sql_string).df()

In [16]:
with duckdb.connect('duck.db') as duckdb_conn:
    stop_ids_sql = ", ".join([f"'{stop_id}'" for stop_id in stops_data_df['stop_id'].to_list()])
    setup_azure_duckdb(duckdb_conn)
    stop_times_df = get_stop_times_from_azure(duckdb_conn, stop_ids_sql)
    logging.info('Stop times data loaded')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [21]:
stop_times_df.to_csv('stop_times.csv', index=False)

In [46]:
stop_times_df = stop_times_df.assign(
    current_stop_departure = pd.to_datetime(stop_times_df['current_stop_departure']),  
    current_stop_arrival = pd.to_datetime(stop_times_df['current_stop_arrival']),
    created_at = pd.to_datetime(stop_times_df['created_at']),
    updated_at = pd.to_datetime(stop_times_df['updated_at']),
    base_stop_id = stop_times_df['gtfs_stop_id'].str.extract(r'^(.*?)(?=[SZ])')[0]
)

In [38]:
stop_times_df.columns

Index(['rt_trip_id', 'gtfs_date', 'gtfs_trip_id', 'gtfs_direction_id',
       'gtfs_route_short_name', 'gtfs_route_type', 'run_number',
       'vehicle_registration_number', 'gtfs_stop_sequence', 'gtfs_stop_id',
       'current_stop_arrival', 'current_stop_departure',
       'current_stop_arr_delay', 'current_stop_dep_delay', 'create_batch_id',
       'created_at', 'created_by', 'update_batch_id', 'updated_at',
       'updated_by', 'origin_route_name', 'day', 'month', 'year'],
      dtype='object')

In [22]:
stop_times_df

,rt_trip_id,gtfs_date,gtfs_trip_id,gtfs_direction_id,gtfs_route_short_name,gtfs_route_type,run_number,vehicle_registration_number,gtfs_stop_sequence,gtfs_stop_id,...,create_batch_id,created_at,created_by,update_batch_id,updated_at,updated_by,origin_route_name,day,month,year
449944,2024-05-31T23:30:00+02:00_94_1867_240428_8278,2024-05-31,94_1867_240428,0,94,0,74,8278.0,23,U100Z4P,...,None,2024-06-01 02:18:00.083005+02:00,None,None,2024-06-01 02:18:00.083005+02:00,None,94,1,6,2024
449945,2024-05-31T23:30:00+02:00_94_1867_240428_8278,2024-05-31,94_1867_240428,0,94,0,74,8278.0,24,U717Z2P,...,None,2024-06-01 02:18:00.083005+02:00,None,None,2024-06-01 02:18:00.083005+02:00,None,94,1,6,2024
449946,2024-05-31T23:30:00+02:00_94_1867_240428_8278,2024-05-31,94_1867_240428,0,94,0,74,8278.0,25,U439Z1P,...,None,2024-06-01 02:18:00.083005+02:00,None,None,2024-06-01 02:18:00.083005+02:00,None,94,1,6,2024
449955,2024-06-01T00:08:00+02:00_97_1401_231206_8363,2024-05-31,97_1401_231206,0,97,0,55,8363.0,14,U163Z1P,...,None,2024-06-01 02:18:00.083005+02:00,None,None,2024-06-01 02:18:00.083005+02:00,None,97,1,6,2024
449951,2024-06-01T00:00:00+02:00_94_1860_240527_8464,2024-05-31,94_1860_240527,0,94,0,61,8464.0,23,U100Z4P,...,None,2024-06-01 02:18:00.083005+02:00,None,None,2024-06-01 02:18:00.083005+02:00,None,94,1,6,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747532,2024-12-31T23:34:05+01:00_993_3643_210715_26_26,2024-12-31,993_3643_210715,0,C,1,26,NaN,7,U100Z102P,...,None,2025-01-01 00:48:00.058105+01:00,None,None,2025-01-01 00:48:00.058105+01:00,None,993,31,12,2024
747512,2024-12-31T23:02:00+01:00_97_1898_241231_9425,2024-12-31,97_1898_241231,1,97,0,61,9425.0,34,U163Z2P,...,None,2025-01-01 00:48:00.058105+01:00,None,None,2025-01-01 00:48:00.058105+01:00,None,22,31,12,2024
747534,2024-12-31T23:41:40+01:00_991_1338_241224_16_16,2024-12-31,991_1338_241224,1,A,1,16,NaN,6,U163Z101P,...,None,2025-01-01 00:48:00.058105+01:00,None,None,2025-01-01 00:48:00.058105+01:00,None,991,31,12,2024
747531,2024-12-31T23:33:35+01:00_991_1339_241223_4_4,2024-12-31,991_1339_241223,0,A,1,4,NaN,12,U163Z102P,...,None,2025-01-01 00:48:00.058105+01:00,None,None,2025-01-01 00:48:00.058105+01:00,None,991,31,12,2024


In [47]:
df

,stop_name,avg_longitude,avg_latitude,base_stop_id,all_stop_ids
0,Chotkovy sady,14.409043,50.095165,U187,"[U187Z1P, U187Z2P]"
1,Hradčanská,14.404189,50.097498,U163,"[U163S1, U163Z1P, U163Z10P, U163Z101P, U163Z10..."
2,Kamenická,14.428636,50.099638,U231,"[U231Z1P, U231Z2P]"
3,Korunovační,14.419715,50.100393,U262,"[U262Z2P, U262Z3P, U262Z4P]"
4,Letenské náměstí,14.423605,50.099975,U324,"[U324Z1P, U324Z2P]"
5,Nábřeží Kapitána Jaroše,14.431299,50.096321,U439,[U439Z1P]
6,Sparta,14.417687,50.099160,U692,"[U692Z1P, U692Z2P, U692Z3P, U692Z4P]"
7,Strossmayerovo náměstí,14.433265,50.098910,U717,"[U717Z1P, U717Z2P, U717Z4P, U717Z5P]"
8,Veletržní palác,14.433064,50.101734,U841,"[U841Z1P, U841Z2P, U841Z4P]"
9,Vltavská,14.438300,50.099185,U100,"[U100S1, U100Z1P, U100Z101P, U100Z102P, U100Z2..."


In [49]:
stop_times_df.merge(df, left_on='stop_id_base', right_on='base_stop_id', how='inner')

,rt_trip_id,gtfs_date,gtfs_trip_id,gtfs_direction_id,gtfs_route_short_name,gtfs_route_type,run_number,vehicle_registration_number,gtfs_stop_sequence,gtfs_stop_id,...,origin_route_name,day,month,year,stop_id_base,stop_name,avg_longitude,avg_latitude,base_stop_id,all_stop_ids
0,2024-05-31T23:30:00+02:00_94_1867_240428_8278,2024-05-31,94_1867_240428,0,94,0,74,8278.0,23,U100Z4P,...,94,1,6,2024,U100,Vltavská,14.438300,50.099185,U100,"[U100S1, U100Z1P, U100Z101P, U100Z102P, U100Z2..."
1,2024-06-01T00:00:00+02:00_94_1860_240527_8464,2024-05-31,94_1860_240527,0,94,0,61,8464.0,23,U100Z4P,...,94,1,6,2024,U100,Vltavská,14.438300,50.099185,U100,"[U100S1, U100Z1P, U100Z101P, U100Z102P, U100Z2..."
2,2024-06-01T00:00:00+02:00_911_490_240428_6741,2024-05-31,911_490_240428,0,911,3,55,6741.0,24,U100Z6P,...,911,1,6,2024,U100,Vltavská,14.438300,50.099185,U100,"[U100S1, U100Z1P, U100Z101P, U100Z102P, U100Z2..."
3,2024-06-01T00:17:00+02:00_905_257_240428_6820,2024-05-31,905_257_240428,0,905,3,56,6820.0,22,U100Z6P,...,905,1,6,2024,U100,Vltavská,14.438300,50.099185,U100,"[U100S1, U100Z1P, U100Z101P, U100Z102P, U100Z2..."
4,2024-06-01T00:06:00+02:00_911_493_240428_6669,2024-05-31,911_493_240428,1,911,3,51,6669.0,19,U100Z5P,...,911,1,6,2024,U100,Vltavská,14.438300,50.099185,U100,"[U100S1, U100Z1P, U100Z101P, U100Z102P, U100Z2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425988,2024-12-31T22:42:00+01:00_91_1980_241231_9430,2024-12-31,91_1980_241231,1,91,0,60,9430.0,30,U231Z2P,...,5,31,12,2024,U231,Kamenická,14.428636,50.099638,U231,"[U231Z1P, U231Z2P]"
2425989,2024-12-31T22:58:00+01:00_91_2004_241231_9281,2024-12-31,91_2004_241231,0,91,0,56,9281.0,20,U231Z1P,...,26,31,12,2024,U231,Kamenická,14.428636,50.099638,U231,"[U231Z1P, U231Z2P]"
2425990,2024-12-31T22:54:00+01:00_96_992_241231_9229,2024-12-31,96_992_241231,1,96,0,60,9229.0,24,U231Z2P,...,6,31,12,2024,U231,Kamenická,14.428636,50.099638,U231,"[U231Z1P, U231Z2P]"
2425991,2024-12-31T23:12:00+01:00_96_980_241231_9301,2024-12-31,96_980_241231,0,96,0,56,9301.0,13,U231Z1P,...,1,31,12,2024,U231,Kamenická,14.428636,50.099638,U231,"[U231Z1P, U231Z2P]"


In [45]:
stop_times_df = stop_times_df.merge(df, on='gtfs_stop_id', right_on='stop_id', how='left')

KeyError: 'stop_id'